In [ ]:
# only 4 anaconda users
# for our project we need to install some packages
# in a lokal IDE  this must be done via console or the anaconda package manager

#Windows
!install https://support.microsoft.com/en-us/help/2977003/the-latest-supported-visual-c-downloads
!install https://visualstudio.microsoft.com/de/thank-you-downloading-visual-studio/?sku=Community&rel=15
!pip install pm4py
!pip install graphviz
!pip install pm4pybpmn
#!pip install pygraphviz

#mac
  # pip install pm4py
  # python -c "import pm4py"
  # pip install graphviz
  # pip install pm4pybpmn
  # pip install pygraphviz


#linux
  # g++ -v
  # apt-get install graphviz
  # pip install pm4py
  # pip install graphviz
  # pip install pm4pybpmn
  # pip install pygraphviz

In [ ]:
#import von Libs für COLAB
if 'google.colab' in str(get_ipython()):
    !pip install pm4py
    !apt-get install graphviz
  #!pip install graphviz
  #!apt-get install python3-tk
  #!apt install libblas-dev
  #!apt install liblapack-dev
  #!apt install libsuitesparse-dev

from IPython.display import Image

# Process Mining

You should already know what process mining is from the lecture. In the following, we will look at the technical side of the basics of process mining, the data and algorithms behind the buzzword. In order to ensure a deeper insight into the topic, we want to provide an understanding of event logs, common process discovery algorithms and forms of visualisation.

## 1. Event log

As already mentioned, event logs are the basis for process mining. Event logs are often not available in operational application systems ( BAS such as ERP / CRM / etc.) in the form in which they are actually required for process mining. In most cases, the event logs first have to be reconstructed from database entries in the IT systems. This reconstruction is often not a simple task, as the syntax, semantics and foreign keys of different tables in a database represent complex relationships.

Process mining solutions such as those from Celonis, which has become known in Germany as the innovative unicorn of IT for its process mining solution, offer ready-made extractors. These enable a technical connection to the systems and the automated creation of event logs from the database tables. However, as you can easily imagine, the accuracy of the automatically extracted logs is linked to the fact that companies have adapted the standard process developed by the IT system manufacturer in the BAS. Adaptations to the process or the BAS can lead to deviations, which means that extractors also have to be adapted.

<img src="https://github.com/JMU-BWL6/IIV_2020_PM/blob/main/1.png?raw=1">

For simplification purposes, the reconstruction and export of such event logs is neglected. In the following, it is therefore assumed that an event log has already been extracted in the form of a file.

In general, the event log can be visualised as a table (see below) in which the activities carried out by users in the IT system are stored together on a case-by-case basis and with a time stamp. Other data can also be included, e.g. to indicate costs and the person who carried out the activity, as in the example.



|Case ID	| Event ID | dd-MM-yyyy:HH.mm |	Activity | Resource | Costs |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 1 | 35654423	| 30-12-2010:11.02	| register request	| Pete	| 50 |
| 1 | 35654424	| 31-12-2010:10.06	| examine thoroughly	| Sue	| 400 |
| 1 | 35654425	| 05-01-2011:15.12	| check ticket	| Mike	| 100 |
| 2 | 35654426	| 06-01-2011:11.18	| decide	| Sara	| 200|
| 2 | 35654427	| 07-01-2011:14.24	| reject request	| Pete	| 200|

### 1.1 Event log formats

Event logs can be available in different file formats. Typical file formats are CSV, XML and XES. While CSV is similar to the table example above and represents the minimum form, XML and XES represent more complex formats in which the data can be grouped and organised. IEEE XES is an XML-based standard format that describes how event logs are stored and prescribes syntactic rules. Further information about the format can be found on the IEEE XES website (http://www.xes-standard.org/).
The following figure shows an example of an XES file.

<img src="https://github.com/JMU-BWL6/IIV_2020_PM/blob/main/2.png?raw=1">

### 1.2 Load event log

This section presents information about importing and exporting event logs stored in different data formats. Before we dive into the details of importing and exporting different types of event data files, we will first briefly explain the two basic concepts of event logs. We assume that the reader is familiar with the general concept of an event log. In general, we distinguish between two different object types of event data:

- **Event streams** (objects.log.log.EventStream); Simply represent a sequence of events. The events themselves are simply an extension of python's mapping class (collections.abc.mapping), which allows events to be used as dictation. From a programming point of view, an event stream behaves exactly like a list object in Python. However, when using Lambda functions, the result must be explicitly cast to an EventStream object.


- **Event Log** (objects.log.log.EventLog); Represents a sequence of event sequences. The concept of an event log is the more traditional view of event data, i.e. the executions of a process are recorded in steps of events, with timestamps as order. In this way, the sorting of traces according to a specific sorting criterion is supported in a natural way, and lambda functions and filters can also be easily applied to event logs.

The sample code shows how an IEEE XES file is imported. The code fragment uses the standard importer. Please note that IEEE XES event logs are imported into an event log object (objects.log.log.Eventlog). The importer converts the file into a Python dict object (https://docs.python.org/3/tutorial/datastructures.html#dictionaries). By adding TIMESTAMP_SORT to the parameters, the event log is sorted according to the timestamp during import.

*Important: the .xes file must be located in the same folder as the Python notebook during execution, otherwise the path to the file must be specified.*

In [ ]:
#get & load LOG
#!pip install wget

!wget https://bitbucket.org/jmu-iiv-2020/iiv_2020_1/raw/fe305beff8605517f799474ccec9fd41598a6ffb/Hospital_log.xes

In [ ]:
from pm4py.objects.log.importer.xes import importer as xes_importer
variant = xes_importer.Variants.ITERPARSE
parameters = {variant.value.Parameters.TIMESTAMP_SORT: True}
log = xes_importer.apply('Hospital_log.xes', variant=variant, parameters=parameters)

In [ ]:
#download a second logfile
!wget https://raw.githubusercontent.com/JMU-BWL6/IIV_2020_PM/main/running-example.xes

In [ ]:
#load the second logfile
from pm4py.objects.log.importer.xes import importer as xes_importer
variant = xes_importer.Variants.ITERPARSE
parameters = {variant.value.Parameters.TIMESTAMP_SORT: True}
log1 = xes_importer.apply('running-example.xes', variant=variant, parameters=parameters)

After execution, 1143 of 1143 activities should have been extracted from the event log.

### 1.3 Exploration

After importing, we want to view an event in the event log. In order to display the latest entry in the event log, the first entry must be read due to the sorting during import.

*Important: Programming languages such as python always start counting at 0 ;)*

In [ ]:
print(log[0][0])

If we want to view the first activity, the last event must be output accordingly.

In [ ]:
print(log[1142][0])

### 1.4 Searching / filtering event data

It is also possible to change the event log. For example, to remove incorrect data or exclude times. The following code removes all timestamps outside the years 2007 and 2008.

In [ ]:
from pm4py.algo.filtering.log.timestamp import timestamp_filter
log_filtered = timestamp_filter.filter_traces_contained(log, "2007-01-01 00:00:00", "2008-12-31 23:59:59")

In [ ]:
len(log_filtered)

After filtering, there are only 261 events in the log.

## 2. Process Discovery

***Process discovery algorithms*** are designed to create a suitable process model from the data that describes the sequence of events/activities that were carried out during a process execution.

The following overview provides an insight into various process mining algorithms used for process discovery.


| Alpha | Alpha+ | Heuristic | Inductive |
| :--- | :--- | :--- | :--- |
| Kann keine Schleifen der Länge eins und zwei handhaben | Kann keine Schleifen der Länge eins und zwei handhaben | Berücksichtigt die Häufigkeit | Kann mit unsichtbaren Aufgaben umgehen |
| Unsichtbare und doppelte Aufgaben können nicht entdeckt werden |  Unsichtbare und doppelte Aufgaben können nicht entdeckt werden | Erkennt kurze Schleifen | Modell ist solide |
|Entdecktes Modell ist möglicherweise nicht ausgereift | Entdecktes Modell ist möglicherweise nicht ausgereift | Garantiert kein fundiertes Modell | Meist verwendeter Algorithmus zur Prozessanalyse
| Schwach gegen fehler im Log | Schwach gegen fehler im Log |    |

### 2.1 Flow graph

**Directly-follows diagrams** are presented in this section. Directly-follows graphs are graphs where the nodes represent the events/activities in the log and there are directed edges between the nodes if there is at least one trace in the log where the source event/activity is followed by the target event/activity. Beyond these directed edges, it is easy to represent metrics such as frequency (counting how often the source event/activity is followed by the target event/activity) or performance (some aggregation, e.g. the average of the time elapsed between the two events/activities).

First, the directly-follows diagram is extracted from the event log already used above. In addition, code is provided to visualise the directly-follows graph. This visualisation is a coloured representation of the directly-follows graph, decorated with the frequency of the activities.

In [ ]:
#Import of required libraries
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization

#Build Flow Graph
dfg = dfg_discovery.apply(log1, variant=dfg_discovery.Variants.PERFORMANCE)

#Export as PNG
parameters = {dfg_visualization.Variants.PERFORMANCE.value.Parameters.FORMAT: "png"}
gviz = dfg_visualization.apply(dfg, log=log1, variant=dfg_visualization.Variants.PERFORMANCE, parameters=parameters)
dfg_visualization.save(gviz, "1_log1.png")

# Visualisation in Jupyter Notebook
gviz = dfg_visualization.apply(dfg, log=log1, variant=dfg_visualization.Variants.FREQUENCY)
dfg_visualization.view(gviz)



### 2.2 Heuristics

The Heuristics Miner is an algorithm based on the Directly Follows graphs and provides a way to deal with noise and find common constructs (i.e. dependency between two activities). The output of the Heuristics Miner is a heuristic net, i.e. an object that contains the activities and the relationships between them. The heuristics network can then be converted into a Petri net. The algorithm was developed by the research group of Wil van der Aalst (https://pdfs.semanticscholar.org/1cc3/d62e27365b8d7ed6ce93b41c193d0559d086.pdf).

In [ ]:
# Import of required libraries
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer

# Generate heuristic network
heu_net = heuristics_miner.apply_heu(log, parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.99})


In [ ]:
#Export as PNG
parameters = {dfg_visualization.Variants.PERFORMANCE.value.Parameters.FORMAT: "png"}
gviz = hn_visualizer.apply(heu_net , parameters=parameters)
hn_visualizer.save(gviz, "2_heu_net.png")

# Visualisation in Jupyter Notebook
hn_visualizer.view(gviz)

*Note*: The output can be zoomed by double-clicking. The output is also saved as a file in the file path in which the Jupyter Notebook is saved.

### 2.3 IMDF

IMDF is a specific implementation of the **Inductive Miner Directly Follows algorithm** that aims to construct a solid workflow network with good fitness values (in most cases to ensure perfect replayability). The basic idea of Inductive Miner is to recognise a "cut" in the log (e.g. sequential cut, parallel cut, concurrent cut and loop cut) and then repeat it in sub-logs found when applying the cut until a base case is found. The Directly Follows variant avoids recursion in the sub-protocols, but uses the Directly Follows graph.

Before we start, we want to import a new example from an event log.

In the following, first the decision tree and then the workflow graph are constructed.

In [ ]:
# Import of IMDF libraries for Process Trees
import os
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.visualization.process_tree import visualizer as pt_visualizer

In [ ]:
# Generate process tree using IMDF
tree = inductive_miner.apply(log1)

In [ ]:
# Output of the tree in the Jupyter Notebook
gviz = pt_visualizer.apply(tree)
pt_visualizer.view(gviz)

# Export to file
parameters = {dfg_visualization.Variants.PERFORMANCE.value.Parameters.FORMAT: "svg"}
gviz = pt_visualizer.apply(tree, parameters=parameters)
pt_visualizer.save(gviz, "3_imdf_tree.svg")

# Warnings can be ignored

In [ ]:
# Output only relevant/execute for colab
from IPython.display import SVG, display
display(SVG('3_imdf_tree.svg'))

In [ ]:
# Import of IMDF libraries for Petrinet
import os
import pm4py
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.visualization.petri_net import visualizer as pn_visualizer


In [ ]:
# Generate Net
net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(log1)

# Output of the net in Jupyter Notebook
pm4py.view_petri_net(net, initial_marking, final_marking)

# Export as  file
pm4py.write_pnml(net, initial_marking, final_marking, "4_imdf_petri.pnml")

## 3. Additional Exercises

The following tasks help to deepen and apply the knowledge.
You can use the documentation to help you to solve the tasks: https://pm4py.fit.fraunhofer.de/documentation

### 3.1 Generate a BPMN diagram for the log1 event.


In this task, we use the data set that we already analysed in the process discovery section (log1).

*Note: the event log should be converted directly from a process tree into a BPMN (Consider 2.3 IMDF)*.

In [ ]:
# Generate process tree using IMDF


In [ ]:
# Generate bpmn graph
bpmn_graph = pm4py.convert_to_bpmn(____)

In [ ]:
# visualise bpmn graph
pm4py.view_bpmn(____)

### 3.2 Identification and visualisation of the process flow for a specific case

In the following, we would like to extract the process for one case (case_id = 4) from our created Petrinet for log1:

In this scenario, you can use a dataframe to filter the desired case.

In [ ]:
import pandas as pd
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

# Convert the process log (log1) into a Pandas DataFrame.
df = pm4py.convert_to_dataframe(log1)

# Filter the process of case 4 of the created dataframe
df_case = df[df['case:concept:name'] == '4']

# Convert the DataFrame (df_case) into a process log (case_log).
case_log = log_converter.apply(df_case)

#### 3.2.1 Create Petri net for the filtered case.

In [ ]:
# Create a Petri net for the process of case 4
case_net, case_initial_marking, case_final_marking = pm4py.discover_petri_net_inductive(____)

In [ ]:
from pm4py.visualization.petri_net import visualizer as pn_visualizer

# Visualise the Petri net
gviz = pn_visualizer.apply(____________________________)
pn_visualizer.view(____)

#### 3.2.2 Create a bpmn for the extracted case

In [ ]:
# Hint: you can also use the df_case dataframe to generate the bpmn model


### 3.3 Directly-follows diagram and heuristic network

For this task, download the **receipt.xes** file from Wuecampus.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:

file_path = '/content/drive/MyDrive/EX_11_PM/receipt.xes'

# Load the XES file into your Colab
import pm4py
log2 = pm4py.read_xes(file_path, return_legacy_log_object=True)


#### 3.3.1 create a Directly-follows diagram for the loaded log

In [ ]:
# Import of required libraries
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization

#Build Flow Graph


#Export as PNG


# Visualisation in Jupyter Notebook



#### 3.3.2 How many events are in the log?

#### 3.3.3 Generate a heuristic network for the log and visualize it.

In [ ]:
# Import of required libraries
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer

# Generate heuristic network


In [ ]:
# Export as PNG


# Visualisation in Jupyter Notebook


## 4. Conclusion

With the examples given, the processing of event logs in the field of process mining, common forms of representation and widespread process discovery algorithms were demonstrated. If you are interested in the topic, you can of course try it out yourself, you will find everything you need via the sources. Have fun!

##  Sources
1. https://pm4py.fit.fraunhofer.de/documentation
2. https://data.4tu.nl/
3. http://www.processmining.org/_media/blogs/pub2015/bpmds_directly-follows_mining.pdf
4. https://pdfs.semanticscholar.org/1cc3/d62e27365b8d7ed6ce93b41c193d0559d086.pdf